Now we have the answers which are close to the query asked, and they might be many. </br>
So we take all these answered, feed it to the LLM, which will give us the single answer to the question.

In [ ]:
pip install streamlit

In [ ]:
pip install langchain langchain-openai faiss-cpu 

In [ ]:
# Run this in a new cell at the top of your notebook
%pip install langchain langchain-classic langchain-openai langchain-community langchain-text-splitters faiss-cpu

In [13]:
import os
import streamlit as st
import pickle
import time
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader

# 1. Load the Partner Package for OpenAI (Matches langchain-openai 1.1.0)
from langchain_openai import OpenAI, OpenAIEmbeddings

# 2. Load the Community Tools (Matches langchain-community 0.4.1)
from langchain_community.document_loaders import TextLoader
# TextLoader cannot read websites. You need UnstructuredURLLoader.
from langchain_community.document_loaders import UnstructuredURLLoader

from langchain_community.vectorstores import FAISS

# 3. Load the Splitter (Matches langchain-text-splitters 1.0.0)
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 4. Load the Chains (Matches langchain 1.1.0)
from langchain_classic.chains import RetrievalQAWithSourcesChain
from langchain_classic.chains.qa_with_sources.loading import load_qa_with_sources_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

### 1. Setup the Brain (The LLM)

In [15]:
llm = OpenAI(temperature=0.9, max_tokens=500)

In [16]:
os.environ["USER_AGENT"] = "MyBot/1.0 (Contact: my-email@example.com)"
loader = UnstructuredURLLoader(urls=["https://www.addcomposites.com/post/overview-of-automated-fiber-placement-process",
                                     "https://www.asg.ed.tum.de/en/lcc/chair-structure/automated-fiber-placement/",
                                     ])
data = loader.load()
len(data)

2

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) 
docs = text_splitter.split_documents(data)
len(docs) 

85

In [18]:
embeddings = OpenAIEmbeddings()
vectorIndex = FAISS.from_documents(docs, embedding=embeddings)

In [19]:
vectorIndex

In [20]:
file_path = "vectorindex_openai.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorIndex, f)

if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)    

TypeError: cannot pickle '_thread.RLock' object

In [22]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question.\nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x174aaaf90>, async_client=<openai.resources.completions.AsyncCompletions object at 0x174aab8c0>, temperature=0.9, max_tokens=500, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, parti

In [ ]:
import langchain
from langchain_core.callbacks import StdOutCallbackHandler
handler = StdOutCallbackHandler()

#langchain.debug = True  # this doesn't work
query = "What is Automated Fiber Placement (AFP)?"

result = chain.invoke({"question": query}, config={"callbacks": [handler]})
result



> Entering new RetrievalQAWithSourcesChain chain...


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return any relevant text verbatim.
Overview of Automated Fiber Placement Process

August 20, 2024

This is some text inside of a div block.



1. Introduction

In the realm of composite material manufacturing, the quest for precision, efficiency, and innovation has led to the development of groundbreaking technologies that redefine traditional processes. Among these, Automated Fiber Placement (AFP) stands out as a transformative approach, offering a leap forward in the production of complex, high-performance parts. This technology, which melds the precision of automation with the flexibility of composite materials, has not only revolutionized the way we think about manufacturing but also opened up new avenues for design

{'question': 'What is Automated Fiber Placement (AFP)?',
 'answer': ' Automated Fiber Placement (AFP) is an automated manufacturing process for composite components that offers unparalleled precision, efficiency, and quality.\n',
 'sources': 'https://www.asg.ed.tum.de/en/lcc/chair-structure/automated-fiber-placement/, https://www.addcomposites.com/post/overview-of-automated-fiber-placement-process'}